# Create SparkSession, import packages

### To update git hub
### git add *
### git commit -m "Insert you change message"
### git push origin master

# To Do:
* combine the data with the features file, to get a header
* do variable selection and transformation
* spit out a final `.csv` to use for modeling

# Ideas
* look at the data via kmeans with 6 clusters?
* compute AUC on each variable to see which are important? (after regressing on the var)
* create variables that are combinations of other variables
* look for outliers and then run pca before clustering?
* run multinomial regression with elastic net? (with k-fold cv)
* or maybe try neural nets?
* something time series?


In [1]:
from pyspark.sql import SparkSession, SQLContext
import pyspark.sql.types as typ
import pyspark.sql.functions as F
import os
import pandas as pd
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("xor") \
    .config("spark.executor.memory", '2g') \
    .config('spark.executor.cores', '1') \
    .config('spark.cores.max', '1') \
    .config("spark.driver.memory",'1g') \
    .getOrCreate()

sc = spark.sparkContext
sqlCtx = SQLContext(sc)

In [2]:
#read in with pandas first since the data is weirdly delimited
pandas_train_features = pd.read_csv("Data/X_train.txt", sep='\s+',header=None)
#conversion to spark df
df_train_features = sqlCtx.createDataFrame(pandas_train_features)

In [3]:
#training labels
pandas_train_labels = pd.read_csv("Data/y_train.txt", sep='\s+',header=None)
pandas_train_labels.columns=['position']
#conversion to spark df
df_train_labels = sqlCtx.createDataFrame(pandas_train_labels)

In [4]:
#test features
pandas_test_features = pd.read_csv("Data/X_test.txt", sep='\s+',header=None)
#conversion to spark df
df_test_features = sqlCtx.createDataFrame(pandas_test_features)

In [5]:
#test labels
pandas_test_labels = pd.read_csv("Data/y_test.txt", sep='\s+',header=None)
pandas_test_labels.columns=['position']
#conversion to spark df
df_test_labels = sqlCtx.createDataFrame(pandas_test_labels)

In [6]:
#header
pandas_header = pd.read_csv("Data/features.txt", sep='\s+',header=None)
#conversion to spark df. this has an index column, so we deselect it
df_header = sqlCtx.createDataFrame(pandas_header)


In [7]:
#reading subject number in
panda_sub_train=pd.read_csv("Data/subject_train.txt",sep='\s+',header=None)
panda_sub_train.columns=['id']
panda_sub_test=pd.read_csv("Data/subject_test.txt",sep='\s+',header=None)
panda_sub_test.columns=['id']
df_sub_train=sqlCtx.createDataFrame(panda_sub_train)
df_sub_test=sqlCtx.createDataFrame(panda_sub_test)

In [8]:
#merge x feature files with subject id
merged_train=pd.merge(panda_sub_train,pandas_train_features, right_index=True, left_index=True)
merged_test=pd.merge(panda_sub_test,pandas_test_features, right_index=True, left_index=True)
df_merged_train_features=sqlCtx.createDataFrame(merged_train)
df_merged_test_features=sqlCtx.createDataFrame(merged_test)


In [17]:
#merge train files with subject id, testing and training sets
#pd.concat([df_a, df_b], axis=1) example
mergetrain=pd.concat([merged_train,pandas_train_labels],axis=1)
mergetest=pd.concat([merged_test,pandas_test_labels],axis=1)
combine=pd.concat([mergetrain,mergetest],axis=0)

In [18]:
df_complete_merge=sqlCtx.createDataFrame(combine)

In [19]:
(train_data2, test_data2)=df_complete_merge.randomSplit(seed=314,weights=[0.8,0.2])

In [11]:
from pyspark.sql.functions import monotonically_increasing_id 
testing=df_merged_train_features.withColumn('index',monotonically_increasing_id())
testing_labels=df_train_labels.withColumn('index',monotonically_increasing_id())
#out=testing.join(test_labels,test.index=testing_labels.index,how='outer')
#merge test files

SyntaxError: keyword can't be an expression (<ipython-input-11-44ddd3b1d535>, line 4)

In [ ]:
#reading in y responses for training
pandas_y_train = pd.read_csv("Data/y_train.txt",header=None)
df_y_train = sqlCtx.createDataFrame(pandas_y_train)


#reading in y responses for test
pandas_y_test = pd.read_csv("Data/y_test.txt",header=None)
df_y_test = sqlCtx.createDataFrame(pandas_y_test)              #Kevin


#is there a reason we read this in twice? this data is already in pandas_train_labels/pandas_test labels, no? -- Nathan

In [ ]:
df_merged_train_features.show()

In [ ]:
#check for missing values
from pyspark.sql.functions import isnan, when, count, col
df_merged_train_features.select([count(when(isnan(c), c)).alias(c) for c in df_merged_train_features.columns]).show()


In [ ]:
df_merged_train_features.select('0','1','2','3','4').describe().show()

In [ ]:
df_merged_train_features.select('0','1','2','3','4').describe().show()

In [ ]:
ds = pd.DataFrame(index=[1], columns=['train_features_obs','train_labels_obs','test_features_obs','test_labels_obs', 'header_obs'])
ds['train_features_obs'] = df_train_features.count()
ds['train_labels_obs'] = df_train_labels.count()
ds['test_features_obs'] = df_test_features.count()
ds['test_labels_obs'] = df_test_labels.count()
ds['header_obs']= df_header.count()
ds=sqlCtx.createDataFrame(ds)
ds.show()                #created data frame to show how many obs in each file
                        #makes it easier to see under df, -Kevin


    


In [ ]:
#splits the header column on dashes, and creates a data frame with each of the elements

split_col = F.split(df_header['1'], '-')
df_header = df_header.withColumn('Main', split_col.getItem(0))
df_header = df_header.withColumn('Seconded', split_col.getItem(1)) #Kevin

In [ ]:
df_header.show()

In [ ]:
#count the number of unique elements in the "main" column of the header df
header_summary=df_header.groupBy('Main').count()
header_summary.count()                               #Kevin
              

In [ ]:
#show the unique elements in "main" by count (show all 24 of them)
header_summary.orderBy('count',ascending = False).show(24)      #Kevin

#should we combine angles? - nathan

In [ ]:
#separate data into the multiple data frames, one for each row above

#step 0: try to pull out and combine angles
header_summary.where(col('Main').like("%angle%")).show()


#step 1: create vector with the "group names" as entries
header_summary.col('Main')

In [ ]:
header_summary.groupBy("Main").count().rdd.values().histogram(20)   #Kevin

In [ ]:
import matplotlib.pyplot

header_barplot= df_header.toPandas()                    # converted to pandas since easier, RDD tricky
header_barplot['Main'].value_counts().plot(kind='bar')  #Visualize Main Variable count
                                                        #Kevin

In [ ]:
#visualizing subject activity count 

In [ ]:
#count the number of activites done
y_train_summary=df_y_train.groupBy('0').count()       #0 is column name
y_train_summary.count()     # total activities                                                  #kevin

In [ ]:
#look for class imbalance in how much each activity was done.
y_train_summary.show()

In [ ]:
y_train_summary_dist = y_train_summary.orderBy('count', ascending = False)
y_train_summary_dist.show()

In [ ]:
# add a column for proportion
sum_counts = df_train_labels.count()
y_train_summary_dist.withColumn('Frequency (%)', F.round(y_train_summary_dist[1]/sum_counts * 100, 2)).show()


In [ ]:
y_train_barplot= df_y_train.toPandas()                    # converted to pandas since easier, RDD tricky
y_train_barplot['0'].value_counts().plot(kind='bar')      #maybe nicer if we change x axis to display activity label?

In [ ]:
y_test_summary=df_y_test.groupBy('0').count()       #0 is column name
y_test_summary.count()     # total activities 

In [ ]:
import matplotlib.pyplot
y_train_bar=y_train_barplot['0'].value_counts() #data read
increment=[1,2,3,4,5,6]
names=['laying','standing','sitting','walking','walking\ndownstairs','walking\nupstairs']
matplotlib.pyplot.bar(increment, y_train_bar, align='center', alpha=0.5, color='c') #Barplot
matplotlib.pyplot.xticks(increment,names)
matplotlib.pyplot.ylabel('Counts')
matplotlib.pyplot.xlabel('Positions')
matplotlib.pyplot.title('Acitivity position distribution')
matplotlib.pyplot.show()

#how are the positiosn distributed?

In [ ]:
y_test_summary.show()

In [ ]:
y_test_summary.orderBy('count', ascending=False).show()      #ordered from biggest to smallest
                                                            # kevin

In [ ]:
y_test_barplot= df_y_test.toPandas()                    # converted to pandas since easier, RDD tricky
y_test_barplot['0'].value_counts().plot(kind='bar')     #Kevin
Y_test_bar=y_test_barplot['0'].value_counts()

In [ ]:
increment=[1,2,3,4,5,6]
names=['laying','standing','sitting','walking','walking\ndownstairs','walking\nupstairs']
matplotlib.pyplot.bar(increment, Y_test_bar, align='center', alpha=0.5, color='c') #Barplot
matplotlib.pyplot.xticks(increment,names)
matplotlib.pyplot.ylabel('Counts')
matplotlib.pyplot.xlabel('Positions')
matplotlib.pyplot.title('Acitivity position distribution')
matplotlib.pyplot.show()


# Outlier Detection, Using Interquartile Range Method

In [ ]:
cols = [c for c in df_train_features.columns]   # exclude id from features
bounds = {} # will store lower and upper bounds for each feature              #Will use later, -Kevin

In [ ]:
import pyspark.sql as sparksql
histogram = df_train_features.select('539').rdd.flatMap(lambda x: x).histogram(11)

# Loading the Computed Histogram into a Pandas Dataframe for plotting
pd.DataFrame(
    list(zip(*histogram)), 
    columns=['bin', 'frequency']
).set_index(
    'bin'
).plot(kind='bar');

#539 is fBodyBodyGyroMag-meanFreq()
#histogram for distributiono of a variable
#kevin- uhh kind of junk rn but may be usefull later


# Look at class imbalance in our data